# 3.2. Querying an encrypted model
**protecting privacy and IP simultaneously**


### Set up

In [ ]:
# Import dependencies
import grid as gr
from grid import syft as sy
import torch as th
import skin_cancer_model_utils as scmu

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# Hook torch
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
grid_server = gr.WebsocketGridClient(hook, "http://localhost:3001", id="grid_server")
patient_server = gr.WebsocketGridClient(hook, "http://localhost:3000", id="patient_server")
hospital_server = gr.WebsocketGridClient(hook, "http://localhost:3002", id="hospital_server")
crypto_provider = gr.WebsocketGridClient(hook, "http://localhost:3003", id="crypto_provider")

grid_server.connect()
patient_server.connect()
hospital_server.connect()
crypto_provider.connect()

# Connect nodes to each other
gr.connect_all_nodes([grid_server, patient_server, hospital_server, crypto_provider])

### Query model
#### Get a copy of the encrypted model

In [ ]:
encrypted_model = grid_server.download_model("skin-cancer-model-encrypted")

**Encrypted data for running through the model**

In [ ]:
data, target = scmu.get_data_sample()

In [ ]:
x_sh = data.encrypt(patient_server, hospital_server, crypto_provider=crypto_provider)

#### Run encrypted inference

In [ ]:
%%time
print(encrypted_model(x_sh).request_decryption(), target)